# Notes

We are using pressure gauge as a proxy to see if the clot has disolved already or not. Mass and other variables has been used in the pass too.

Question/To-Do:
-What should be the dimensions of the microbubble and floor (circumference of blood vessel)?
-Incorporate streaming.
-How long does it take to reach terminal velocity?

# Acoustic Radiation Pressure and Stokes Drag

In [1]:
import math

#Parameters
P_a = 150000 #acoustic pressure amplitude in Pa
R_0 = 0.0000022 #equilibrium radius of the bubble in m
c = 1500 #speed of sound in water in m/s
del_tot = 0.1 #dimensionless damping coefficient
kap = 1.07 #dimensionless polytropic index
p_0 = 3333.05968538 #hydrostatic liquid pressure outside the bubble in Pa
rho = 1060 #density of the fluid surrounding the bubble in kg/m^3
mu = 0.0035 #liquid's shear viscosity in Pa*s
V = 0.0175 #bubble velocity relative to the liquid in m/s

#NOTE: F_arf is a time-averaged force. This formula is only correct at resonance frequency.
F_arf = (2*math.pi*(P_a**2)*(R_0**2))/(c*del_tot*math.sqrt(3*kap*p_0*rho))

#NOTE: F_drag is a time-averaged force. This formula is only correct for low Reynolds number.
F_drag = 4*math.pi*mu*V*R_0

#Terminal velocity (Obtained by equating F_arf and F_drag and solving for v)
V_ter = (P_a**2*R_0)/(2*c*mu*del_tot*math.sqrt(3*kap*p_0*rho))

print('Acoustic radiation force:',F_arf)
print('Stokes drag:',F_drag)
print('Terminal velocity:',V_ter)



##########-----NOTES ON PARAMETERS-----##########
#PARAMETERS
#P_a, acoustic pressure amplitude in Pa
#R_0, equilibrium radius of the bubble in m
#c, #speed of sound in water in m/s
#del_tot, #dimensionless damping coefficient:Here, del_tot = del_rad + del_th + del_ vis
#kap, #dimensionless polytropic index: This is from Doinikov et al "Resonance frequencies of lipid-shelled microbubbles...", page 4
#p_0, #hydrostatic liquid pressure outside the bubble in Pa: We can assume 25 mm Hg, where 1 mm Hg = 133.322387415 Pa. Should we use 1 ATM = 101325 Pa instead?
#rho, #density of the fluid surrounding the bubble in kg/m^3
#mu, #liquid's shear viscosity in Pa*s: It is 3 to 4 mPa*s for blood, and 1 mPA*s for water.
#V, #bubble velocity relative to the liquid in m/s: The range is 0.5-3.0 cm/s for Definity at p^-=0.346 MPa^-.
    

Acoustic radiation force: 1.3545327031597484e-06
Stokes drag: 1.6933184402848988e-09
Terminal velocity: 13.998738655032525


# Simulation: Sphere in free-fall

In [5]:
from vpython import *
import math
import numpy as np
scene = canvas(title='Sphere in free-fall',width=600,height=600,center=vector(0,20,0)) #initialize vpython window

g = 9.81 #gravitational acceleration in m/s^2 
m = 1 #mass of the ball in kg
p0 = 200 #initial height in m
v0 = 0 #initial velocity in m/s

ball = sphere(pos=vector(0,p0,0), radius=5) #initialize ball
floor = box(pos=vector(0,0,0), size=vector(200,0.25,200)) #initialize floor

for t in np.linspace(0,10,100): #time in s (how many seconds the ball is in the air)
    rate(10)
    pt = p0 + v0*t - g*t**2/2.0
    vt = v0 - g*t #NOTE: Velocity is negative in the downward direction (direction of the vector)
    if pt <= 0: #if it reaches or goes through the floor, stop
        pt = 0
        vt = v0 - g*((v0+math.sqrt(v0**2+2*g*p0))/(g)) #This is time at pt = 0
        ball.pos.y = pt
        print('Position:',pt)
        print('Instantaneous velocity:',-vt)
        print('Time it touches the floor:',(v0+math.sqrt(v0**2+2*g*p0))/(g))
        break
    ball.pos.y = pt
    print('Position:',pt)
    print('Instantaneous velocity:',-vt)

print('End of program')

<IPython.core.display.Javascript object>

 database. History logging moved to new session Position: 200.0
Instantaneous velocity: -0.0
274
Position: 199.949954086
Instantaneous velocity: 0.990909090909
Position: Position: 198.748852158
Instantaneous velocity: 4.95454545455
Position: 198.198347107
Instantaneous velocity: 5.94545454545
Position: 191.542240588
Instantaneous velocity: 12.8818181818
Position: 187.188246097
Instantaneous velocity: 15.8545454545
Position: 185.536730946
Instantaneous velocity: 16.8454545455
Position: 183.785123967
Instantaneous velocity: 17.8363636364
Position:Position:Position: 173.525711662
Instantaneous velocity:Position: Position: 163.516528926
Instantaneous velocity:Position: 151.905876951
Instantaneous velocity: 30.7181818182
Position: 135.140495868
Instantaneous velocity: 35.6727272727
Position: 131.487144169
Instantaneous velocity: 36.6636363636
Position: 103.111111111
Instantaneous velocity: 43.6
Position: 98.6570247934
Instantaneous velocity: 44.5909090909
Position: 79.8397612489
Instantaneo

# Simulation: Bubble moved by ARF through blood clot

In [1]:
from vpython import *
import numpy as np
scene = canvas(title='Bubble moved by ARF through blood clot',width=600,height=600,center=vector(0,10,0))

a = 1.3545327031597484e-03/3.3e-8 #acceleration due to F_arf in mm/μs^2
m = 3.3e-14 #mass of the bubble in kg (NOTE: Using RBC for now; each blood cell has 33 pg of mass)
p0 = 25 #initial height in mm
v0 = 0 #initial velocity in mm/μs
t = 0 #time in s

bubble = sphere(pos=vector(0,p0,0), radius=0.005) #Dimensions of the bubble
floor = box(pos=vector(0,0,0), size=vector(5,0.00625,5)) #Dimensions of the bubble veins/artery (-ish; also, is this in μm?)

for t in np.linspace(0,1,1000000): #time in s (how many seconds the bubble is moving)
    rate(10)
    pt = p0 + v0*t - a*t**2/2.0
    vt = v0 - a*t #NOTE: Velocity is negative in the downward direction (direction of the vector)
    if pt <= 0: #if it reaches or goes past the end, stop
        pt = 0
        vt = v0 - a*((v0+math.sqrt(v0**2+2*a*p0))/(a)) #This is time at pt = 0
        bubble.pos.y = pt
        print('Position:',pt)
        print('Instantaneous velocity:',-vt)
        print('Time it touches the floor:',(v0+math.sqrt(v0**2+2*a*p0))/(a))
        break
    bubble.pos.y = pt
    print('Position:',pt)
    print('Instantaneous velocity:',-vt)

print('End of program')

ModuleNotFoundError: No module named 'vpython'